In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as stat
import statsmodels.tsa.stattools as ts

In [2]:
#Functions needed for cointegration test and statistical tests
limit=5000
res=3600
tf="1h"
def get_price(tick,res):
    res=str(res)
    hist=requests.get('https://ftx.com/api/markets/'+tick+'-PERP/candles?resolution='+res+'&start_time='+'0'+'&limit='+str(limit)).json()
    hist=pd.DataFrame(hist['result'])
    hist.drop(['startTime'], axis = 1, inplace=True)
    return hist

def get_time(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    aux.set_index('time', inplace=True)
    return aux

def get_initime(hist):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][0]

def get_time(hist,i):
    aux=hist.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    return aux['time'][i]
def coint_test(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    if not get_initime(B)==get_initime(A):
        print('Different initial time')
        return 
    X = np.stack((np.log10(A['close']), np.log10(B['close'])), axis=0)
    cov=np.cov(X)
    #print(cov)
    covAB=cov[0,1]
    varA=cov[0,0]
    varB=cov[1,1]
    cor=covAB/(np.sqrt(varA*varB))
    lAB=covAB/varB #res=log(A)-lAB*log(B)
    lBA=covAB/varA #res=log(B)-lBA*log(A)
    spreadAB=np.log10(A['close'])-lAB*np.log10(B['close'])
    muAB=np.mean(spreadAB)
    spreadBA=np.log10(B['close'])-lBA*np.log10(A['close'])
    muBA=np.mean(spreadBA)
    return cor,lAB,lBA,muAB,muBA,spreadAB,spreadBA

def print_spread(tick1,tick2,res):
    aux=coint_test(tick1,tick2,res)
    if aux[1]>aux[2]:
        print('We are gonna use A-lAB*B with correlation='+str(aux[0])+' and lambda_AB='+str(aux[1]))
        plt.plot(aux[5]-aux[3])
        return aux[5]
    else :
        print('We are gonna use B-lBA*A with correlation='+str(aux[0])+' and lambda_BA='+str(aux[2]))
        plt.plot(aux[6]-aux[4])
        return aux[6]
def adfuller_test(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    aux=stat.OLS(A['close'],B['close']).fit()
    c_t=ts.adfuller(aux.resid)
    if c_t[0]<=c_t[4]['10%'] and c_t[1]<0.1:
        return True
    else :
        return False
def adfuller_test_log(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    aux=stat.OLS(np.log10(A['close']),np.log10(B['close'])).fit()
    c_t=ts.adfuller(aux.resid)
    if c_t[0]<=c_t[4]['10%'] and c_t[1]<0.1:
        #print(tick1+' and '+tick2+' are cointegrated')
        return True
    else :
        #print(tick1+' and '+tick2+' are not cointegrated')
        return False
    
def coint_test(tick1,tick2,res):
    A=get_price(tick1,res)
    B=get_price(tick2,res)
    aux=result=ts.coint(A['close'],B['close'])

In [3]:
#Scraping all perpetual market names
data=requests.get('https://ftx.com/api/futures').json()
data=data['result']
tick_list_aux=[]
for mark in data:
    tickaux=mark['name']
    #For some reason DMG-PERP doesn't exist in FTX
    if mark['group']=='perpetual' and tickaux!='DMG-PERP':
        tick_list_aux.append(tickaux[:-5])
#Obtaining only tickers that have enough price data
btc=get_price('BTC',res)
t0=get_initime(btc)
tick_list=[]
for tick in tick_list_aux:
    print(tick)
    hist_aux=get_price(tick,res)
    t0_aux=get_initime(hist_aux)
    if t0==t0_aux:
        tick_list.append(tick)

1INCH
AAVE
ADA
AGLD
ALCX
ALGO
ALICE
ALPHA
ALT
AMPL
AR
ASD
ATLAS
ATOM
AUDIO
AVAX
AXS
BADGER
BAL
BAND
BAO
BAT
BCH
BIT
BNB
BNT
BOBA
BRZ
BSV
BTC
BTT
C98
CAKE
CEL
CELO
CHR
CHZ
CLV
COMP
CONV
CREAM
CRO
CRV
CUSDT
CVC
DASH
DAWN
DEFI
DENT
DODO
DOGE
DOT
DRGN
DYDX
EDEN
EGLD
ENJ
ENS
EOS
ETC
ETH
EXCH
FIDA
FIL
FLM
FLOW
FTM
FTT
GALA
GRT
HBAR
HNT
HOLY
HOT
HT
HUM
ICP
ICX
IOTA
KAVA
KIN
KNC
KSHIB
KSM
KSOS
LEO
LINA
LINK
LOOKS
LRC
LTC
LUNA
MANA
MAPS
MATIC
MCB
MEDIA
MER
MID
MKR
MNGO
MTA
MTL
MVDA10
MVDA25
NEAR
NEO
OKB
OMG
ONE
ONT
ORBS
OXY
PAXG
PEOPLE
PERP
POLIS
PRIV
PROM
PUNDIX
QTUM
RAMP
RAY
REEF
REN
RNDR
RON
ROOK
ROSE
RSR
RUNE
SAND
SC
SCRT
SECO
SHIB
SHIT
SKL
SLP
SNX
SOL
SOS
SPELL
SRM
SRN
STEP
STMX
STORJ
STX
SUSHI
SXP
THETA
TLM
TOMO
TONCOIN
TRU
TRX
TRYB
TULIP
UNI
UNISWAP
USDT
VET
WAVES
XAUT
XEM
XLM
XMR
XRP
XTZ
YFI
YFII
ZEC
ZIL
ZRX


In [5]:
#Creation of possible pairs
tick_array=np.array(tick_list)
pair_list=[]
for i in range(len(tick_array)):
    for j in range(i+1,len(tick_array)):
        pair_list.append((tick_array[i],tick_array[j]))

In [6]:
#Cointegration test for all pairs
pair_adf=[]
for pair in pair_list:
    try:
        if adfuller_test(pair[0],pair[1],res):
            print(pair[0]+' and '+pair[1]+' are cointegrated')
            pair_adf.append(pair)
        else:
            print(pair[0]+' and '+pair[1]+' are not cointegrated')
    except:
        pass

1INCH and AAVE are not cointegrated
1INCH and ADA are not cointegrated
1INCH and ALCX are cointegrated
1INCH and ALGO are not cointegrated
1INCH and ALPHA are cointegrated
1INCH and ALT are cointegrated
1INCH and AMPL are cointegrated
1INCH and AR are not cointegrated
1INCH and ASD are cointegrated
1INCH and ATOM are not cointegrated
1INCH and AUDIO are cointegrated
1INCH and AVAX are not cointegrated
1INCH and AXS are not cointegrated
1INCH and BADGER are cointegrated
1INCH and BAL are not cointegrated
1INCH and BAND are not cointegrated
1INCH and BAO are not cointegrated
1INCH and BAT are not cointegrated
1INCH and BCH are not cointegrated
1INCH and BNB are not cointegrated
1INCH and BNT are not cointegrated
1INCH and BRZ are not cointegrated
1INCH and BSV are cointegrated
1INCH and BTC are cointegrated
1INCH and CAKE are not cointegrated
1INCH and CEL are not cointegrated
1INCH and CHZ are cointegrated
1INCH and COMP are not cointegrated
1INCH and CONV are not cointegrated
1INCH and

AAVE and SC are not cointegrated
AAVE and SECO are not cointegrated
AAVE and SHIB are not cointegrated
AAVE and SHIT are not cointegrated
AAVE and SKL are not cointegrated
AAVE and SNX are cointegrated
AAVE and SOL are not cointegrated
AAVE and SRM are not cointegrated
AAVE and SRN are cointegrated
AAVE and STEP are not cointegrated
AAVE and STMX are not cointegrated
AAVE and STORJ are not cointegrated
AAVE and STX are not cointegrated
AAVE and SUSHI are cointegrated
AAVE and SXP are cointegrated
AAVE and THETA are cointegrated
AAVE and TOMO are not cointegrated
AAVE and TRU are not cointegrated
AAVE and TRX are not cointegrated
AAVE and TRYB are cointegrated
AAVE and UNI are not cointegrated
AAVE and UNISWAP are not cointegrated
AAVE and USDT are not cointegrated
AAVE and VET are not cointegrated
AAVE and WAVES are not cointegrated
AAVE and XAUT are not cointegrated
AAVE and XEM are cointegrated
AAVE and XLM are not cointegrated
AAVE and XMR are cointegrated
AAVE and XRP are not coint

ALCX and MTA are cointegrated
ALCX and MTL are cointegrated
ALCX and MVDA10 are not cointegrated
ALCX and MVDA25 are not cointegrated
ALCX and NEAR are not cointegrated
ALCX and NEO are not cointegrated
ALCX and OKB are not cointegrated
ALCX and OMG are not cointegrated
ALCX and ONT are cointegrated
ALCX and ORBS are not cointegrated
ALCX and OXY are not cointegrated
ALCX and PAXG are not cointegrated
ALCX and PERP are cointegrated
ALCX and PRIV are cointegrated
ALCX and PROM are cointegrated
ALCX and PUNDIX are cointegrated
ALCX and QTUM are cointegrated
ALCX and RAMP are not cointegrated
ALCX and RAY are not cointegrated
ALCX and REEF are cointegrated
ALCX and REN are not cointegrated
ALCX and ROOK are cointegrated
ALCX and RSR are cointegrated
ALCX and RUNE are cointegrated
ALCX and SAND are not cointegrated
ALCX and SC are cointegrated
ALCX and SECO are not cointegrated
ALCX and SHIB are not cointegrated
ALCX and SHIT are not cointegrated
ALCX and SKL are cointegrated
ALCX and SNX 

ALPHA and FTM are not cointegrated
ALPHA and FTT are cointegrated
ALPHA and GRT are cointegrated
ALPHA and HBAR are not cointegrated
ALPHA and HNT are not cointegrated
ALPHA and HOLY are not cointegrated
ALPHA and HOT are cointegrated
ALPHA and HT are not cointegrated
ALPHA and HUM are not cointegrated
ALPHA and ICP are cointegrated
ALPHA and IOTA are not cointegrated
ALPHA and KAVA are not cointegrated
ALPHA and KIN are cointegrated
ALPHA and KNC are not cointegrated
ALPHA and KSM are not cointegrated
ALPHA and LEO are not cointegrated
ALPHA and LINA are cointegrated
ALPHA and LINK are not cointegrated
ALPHA and LRC are not cointegrated
ALPHA and LTC are cointegrated
ALPHA and LUNA are not cointegrated
ALPHA and MAPS are not cointegrated
ALPHA and MATIC are not cointegrated
ALPHA and MEDIA are cointegrated
ALPHA and MER are cointegrated
ALPHA and MID are not cointegrated
ALPHA and MKR are not cointegrated
ALPHA and MTA are not cointegrated
ALPHA and MTL are cointegrated
ALPHA and MVDA

AMPL and DASH are cointegrated
AMPL and DAWN are cointegrated
AMPL and DEFI are cointegrated
AMPL and DENT are not cointegrated
AMPL and DODO are not cointegrated
AMPL and DOGE are cointegrated
AMPL and DOT are not cointegrated
AMPL and DRGN are cointegrated
AMPL and EGLD are not cointegrated
AMPL and ENJ are not cointegrated
AMPL and EOS are cointegrated
AMPL and ETC are cointegrated
AMPL and ETH are cointegrated
AMPL and EXCH are cointegrated
AMPL and FIDA are not cointegrated
AMPL and FIL are not cointegrated
AMPL and FLM are cointegrated
AMPL and FLOW are not cointegrated
AMPL and FTM are not cointegrated
AMPL and FTT are cointegrated
AMPL and GRT are cointegrated
AMPL and HBAR are cointegrated
AMPL and HNT are not cointegrated
AMPL and HOLY are not cointegrated
AMPL and HOT are cointegrated
AMPL and HT are not cointegrated
AMPL and HUM are not cointegrated
AMPL and ICP are not cointegrated
AMPL and IOTA are cointegrated
AMPL and KAVA are cointegrated
AMPL and KIN are not cointegra

...

PRIV and RSR are cointegrated
PRIV and RUNE are not cointegrated
PRIV and SAND are not cointegrated
PRIV and SC are cointegrated
PRIV and SECO are not cointegrated
PRIV and SHIB are not cointegrated
PRIV and SHIT are not cointegrated
PRIV and SKL are cointegrated
PRIV and SNX are not cointegrated
PRIV and SOL are not cointegrated
PRIV and SRM are not cointegrated
PRIV and SRN are cointegrated
PRIV and STEP are not cointegrated
PRIV and STMX are cointegrated
PRIV and STORJ are not cointegrated
PRIV and STX are not cointegrated
PRIV and SUSHI are cointegrated
PRIV and SXP are not cointegrated
PRIV and THETA are cointegrated
PRIV and TOMO are cointegrated
PRIV and TRU are cointegrated
PRIV and TRX are not cointegrated
PRIV and TRYB are not cointegrated
PRIV and UNI are cointegrated
PRIV and UNISWAP are not cointegrated
PRIV and USDT are not cointegrated
PRIV and VET are not cointegrated
PRIV and WAVES are not cointegrated
PRIV and XAUT are not cointegrated
PRIV and XEM are cointegrated
PR

RAY and VET are not cointegrated
RAY and WAVES are not cointegrated
RAY and XAUT are not cointegrated
RAY and XEM are not cointegrated
RAY and XLM are not cointegrated
RAY and XMR are not cointegrated
RAY and XRP are not cointegrated
RAY and XTZ are not cointegrated
RAY and YFI are not cointegrated
RAY and YFII are not cointegrated
RAY and ZEC are not cointegrated
RAY and ZIL are not cointegrated
RAY and ZRX are not cointegrated
REEF and REN are not cointegrated
REEF and ROOK are cointegrated
REEF and RSR are not cointegrated
REEF and RUNE are cointegrated
REEF and SAND are not cointegrated
REEF and SC are not cointegrated
REEF and SECO are not cointegrated
REEF and SHIB are not cointegrated
REEF and SHIT are not cointegrated
REEF and SKL are not cointegrated
REEF and SNX are not cointegrated
REEF and SOL are not cointegrated
REEF and SRM are not cointegrated
REEF and SRN are not cointegrated
REEF and STEP are not cointegrated
REEF and STMX are not cointegrated
REEF and STORJ are not c

SC and SHIT are not cointegrated
SC and SKL are cointegrated
SC and SNX are not cointegrated
SC and SOL are not cointegrated
SC and SRM are not cointegrated
SC and SRN are not cointegrated
SC and STEP are not cointegrated
SC and STMX are cointegrated
SC and STORJ are not cointegrated
SC and STX are not cointegrated
SC and SUSHI are not cointegrated
SC and SXP are not cointegrated
SC and THETA are cointegrated
SC and TOMO are not cointegrated
SC and TRU are cointegrated
SC and TRX are cointegrated
SC and TRYB are not cointegrated
SC and UNI are not cointegrated
SC and UNISWAP are not cointegrated
SC and USDT are not cointegrated
SC and VET are cointegrated
SC and WAVES are not cointegrated
SC and XAUT are not cointegrated
SC and XEM are cointegrated
SC and XLM are cointegrated
SC and XMR are cointegrated
SC and XRP are cointegrated
SC and XTZ are not cointegrated
SC and YFI are not cointegrated
SC and YFII are cointegrated
SC and ZEC are cointegrated
SC and ZIL are cointegrated
SC and Z

SRM and ZEC are not cointegrated
SRM and ZIL are not cointegrated
SRM and ZRX are not cointegrated
SRN and STEP are not cointegrated
SRN and STMX are cointegrated
SRN and STORJ are not cointegrated
SRN and STX are not cointegrated
SRN and SUSHI are cointegrated
SRN and SXP are cointegrated
SRN and THETA are cointegrated
SRN and TOMO are cointegrated
SRN and TRU are cointegrated
SRN and TRX are not cointegrated
SRN and TRYB are not cointegrated
SRN and UNI are cointegrated
SRN and UNISWAP are not cointegrated
SRN and USDT are not cointegrated
SRN and VET are not cointegrated
SRN and WAVES are cointegrated
SRN and XAUT are not cointegrated
SRN and XEM are cointegrated
SRN and XLM are not cointegrated
SRN and XMR are not cointegrated
SRN and XRP are not cointegrated
SRN and XTZ are not cointegrated
SRN and YFI are not cointegrated
SRN and YFII are cointegrated
SRN and ZEC are not cointegrated
SRN and ZIL are cointegrated
SRN and ZRX are not cointegrated
STEP and STMX are not cointegrated


TRYB and USDT are not cointegrated
TRYB and VET are not cointegrated
TRYB and WAVES are not cointegrated
TRYB and XAUT are not cointegrated
TRYB and XEM are cointegrated
TRYB and XLM are not cointegrated
TRYB and XMR are not cointegrated
TRYB and XRP are not cointegrated
TRYB and XTZ are not cointegrated
TRYB and YFI are not cointegrated
TRYB and YFII are cointegrated
TRYB and ZEC are not cointegrated
TRYB and ZIL are not cointegrated
TRYB and ZRX are not cointegrated
UNI and UNISWAP are not cointegrated
UNI and USDT are not cointegrated
UNI and VET are not cointegrated
UNI and WAVES are cointegrated
UNI and XAUT are not cointegrated
UNI and XEM are cointegrated
UNI and XLM are not cointegrated
UNI and XMR are cointegrated
UNI and XRP are not cointegrated
UNI and XTZ are not cointegrated
UNI and YFI are not cointegrated
UNI and YFII are cointegrated
UNI and ZEC are not cointegrated
UNI and ZIL are cointegrated
UNI and ZRX are not cointegrated
UNISWAP and USDT are not cointegrated
UNISW

In [7]:
np.savetxt("pair_"+tf+".csv", 
            pair_adf,
           delimiter =", ", 
           fmt ='% s')

In [8]:
#Export pairs in the .csv file from the ADF test
with open("pair_"+tf+".csv", newline='') as f:
    reader = csv.reader(f)
    pair_adf = list(reader)

NameError: name 'csv' is not defined

In [ ]:
pair_info=[]
for tick1,tick2 in pair_adf:
    #For some reason the first character of tick2 is a space
    tick2=tick2[1:]
    try:
        aux=coint_test(tick1,tick2,res)
        pair_info.append((tick1,tick2,aux[0],aux[1],aux[2]))
        print(tick1,tick2,aux[0])
    except:
        pass

In [ ]:
pair_info_df=pd.DataFrame(pair_info)
pair_info_df.columns = ['Ticker1', 'Ticker2','ro','lAB','lBA']
pair_info_df=pair_info_df.sort_values(by=['ro'],ascending=False)
pair_info_df=pair_info_df.reset_index()
pair_info_df.to_csv("pair_ro_"+tf+".csv")

In [ ]:
pdf= pd.read_csv("pair_ro_"+tf+".csv",header=0)
#Filter the ro values
res=900
ro_limit=0.85
pdf_f=pdf[pdf['ro']>ro_limit]

In [ ]:
pair_opt=[]
for i in range(len(pdf_f['Ticker1'])):
    tick1=pdf_f['Ticker1'][i]
    tick2=pdf_f['Ticker2'][i]
    print(i)
    try:
        aux=get_spread(tick1,tick2,res)
        diff=abs(aux[2]-aux[0][len(aux[0])-1])
        pair_opt.append((tick1,tick2,aux[1],diff,aux[3],aux[4]))
    except:
        pass

In [ ]:
po_df=pd.DataFrame(pair_opt)
po_df.columns = ['Ticker1', 'Ticker2','ro','diff','lAB','lBA']
po_df=po_df.sort_values(by=['diff'],ascending=False)
po_df.to_csv("pair_opti_"+tf+".csv")